In [1]:
import geopandas as gpd
import numpy as np

lgas = gpd.read_file("/home/kikeportales/Documentos/Investigacio/Floods//pakistan_aoi.shp").to_crs("EPSG:4326")
lgas_bounds = lgas.bounds
bounds_all = np.min(lgas_bounds.minx), np.min(lgas_bounds.miny), np.max(lgas_bounds.maxx), np.max(lgas_bounds.maxy)
bounds_all

(67.02798521942442, 23.6513383987391, 71.63537980464966, 32.86196175130059)

## Compute existing polygons

In [2]:
import os
from ml4floods.data import utils
from tqdm import tqdm

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/kikeportales/Documentos/Investigacio/Floods/Credentials/ml4cc-general-access_request_pays.json"
fs = utils.get_filesystem("gs://ml4cc_data_lake")

data = []
for meta_file in tqdm(fs.glob("gs://ml4cc_data_lake/0_DEV/1_Staging/operational/EMSR570/*/flood_meta/2022-03-*")):
    meta_file = f"gs://{meta_file}"
    meta = utils.read_pickle_from_gcp(meta_file)
    data.append({"geometry":meta["area_of_interest_polygon"], "ems_code":meta["ems_code"], "aoi_code":meta["aoi_code"], "date":meta["satellite date"]})
data = gpd.GeoDataFrame(data,crs="EPSG:4326")
data

KeyboardInterrupt: 

In [27]:

data[["geometry","aoi_code"]].to_file("/home/gonzalo/Downloads/australia_aois.geojson",driver="GeoJSON")

In [5]:
import itertools
from ml4floods.data.copernicusEMS.activations import generate_polygon
from shapely.geometry import Polygon

import warnings
warnings.filterwarnings('ignore', 'Geometry is in a ', UserWarning)

step = 0.20, 0.20
size = 0.21

pols_all = []
for idx, (x_left,y_bottom) in enumerate(itertools.product(np.arange(bounds_all[0], bounds_all[2], step[0]),
                                                          np.arange(bounds_all[1],bounds_all[3], step[1]))):
    bounds_iter = (x_left, y_bottom, x_left+size, y_bottom+size)
    pol = Polygon(generate_polygon(bounds_iter))
    
    
    # TODO do not include pol if it does not overlap the lgas
    lgas_overlap_polygons = lgas.intersection(pol)
    lgas_overlap_polygons = lgas_overlap_polygons[~lgas_overlap_polygons.is_empty].copy()
    if lgas_overlap_polygons.shape[0] == 0:
        # print(f"{idx} does not overlap the LGAS")
        continue
    area_overlap_lga = lgas_overlap_polygons.area.sum() / pol.area
    
    if area_overlap_lga<=.1:
        continue
    
     #do not include pol if it has high overlap with existing polygons
    data = gpd.GeoDataFrame(columns = ['id','geometry'],geometry='geometry')
    aoi_overlap_polygons = data.intersection(pol)
    aoi_overlap_polygons = aoi_overlap_polygons[~aoi_overlap_polygons.is_empty].copy()
    
    area_overlap = aoi_overlap_polygons.area.sum() / pol.area
    if area_overlap >=.3:
        print(f"Do not include {idx}")
        continue
    
    pols_all.append({"geometry": pol, "name":f"AUTOAOI{idx:04d}"})
    
    
pols_all = gpd.GeoDataFrame(pols_all,crs="EPSG:4326")
print(pols_all.shape)
pols_all.to_file("/home/kikeportales/Documentos/Investigacio/Floods/activations/pakistan/autoaois_pak.geojson",driver="GeoJSON")

(372, 2)


In [4]:
pol.area
existing_aois = .is_empty

SyntaxError: invalid syntax (1438337963.py, line 2)

In [6]:
(153.3362,-28.1347) - (153.6401,-28.1467) - (153.3386,-28.3836)

(153.3362, -28.1347)

In [7]:
lgas

,LGA_CODE21,LGA_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,Area_Ha,Shape_Le_1,Shape_Area,geometry
0,10500,Bayside (NSW),1,New South Wales,AUS,Australia,50.6204,http://linked.data.gov.au/dataset/asgsed3/LGA2...,0.576415,5062.303066,0.576415,0.004937,"POLYGON ((151.14804 -33.92726, 151.14857 -33.9..."
1,10750,Blacktown,1,New South Wales,AUS,Australia,238.8471,http://linked.data.gov.au/dataset/asgsed3/LGA2...,0.741291,23884.819558,0.741291,0.023239,"POLYGON ((150.87088 -33.82386, 150.87077 -33.8..."
2,10900,Blue Mountains,1,New South Wales,AUS,Australia,1431.1442,http://linked.data.gov.au/dataset/asgsed3/LGA2...,3.097879,143114.275049,3.097879,0.139173,"POLYGON ((150.60737 -33.66290, 150.61111 -33.6..."
3,11450,Camden,1,New South Wales,AUS,Australia,201.5278,http://linked.data.gov.au/dataset/asgsed3/LGA2...,0.781939,20152.701238,0.781939,0.019668,"POLYGON ((150.76613 -33.94514, 150.76631 -33.9..."
4,11500,Campbelltown (NSW),1,New South Wales,AUS,Australia,311.4117,http://linked.data.gov.au/dataset/asgsed3/LGA2...,0.992642,31141.228732,0.992642,0.030418,"POLYGON ((150.81663 -34.17324, 150.80916 -34.1..."
5,11570,Canterbury-Bankstown,1,New South Wales,AUS,Australia,110.2374,http://linked.data.gov.au/dataset/asgsed3/LGA2...,0.710778,11023.444889,0.710778,0.010748,"POLYGON ((151.03941 -33.88770, 151.03955 -33.8..."
6,11650,Central Coast (NSW),1,New South Wales,AUS,Australia,1681.0059,http://linked.data.gov.au/dataset/asgsed3/LGA2...,5.261147,168100.901593,5.261147,0.162751,"MULTIPOLYGON (((151.31496 -33.55579, 151.31496..."
7,11720,Cessnock,1,New South Wales,AUS,Australia,1965.1593,http://linked.data.gov.au/dataset/asgsed3/LGA2...,2.982206,196515.947807,2.982206,0.189450,"POLYGON ((151.15081 -32.81309, 151.15207 -32.8..."
8,12700,Dungog,1,New South Wales,AUS,Australia,2249.9983,http://linked.data.gov.au/dataset/asgsed3/LGA2...,2.627830,224999.771143,2.627830,0.215545,"POLYGON ((151.44728 -32.39912, 151.44610 -32.3..."
9,12850,Fairfield,1,New South Wales,AUS,Australia,101.5072,http://linked.data.gov.au/dataset/asgsed3/LGA2...,0.652562,10150.600922,0.652562,0.009890,"POLYGON ((150.95623 -33.85769, 150.95631 -33.8..."
